# Machine Learning Madness 2021
**Stage 1**: Use historical data to build and train models  
**Stage 2**: Predict 2021 NCAA champtionship  
**Evaluated by**: Log loss  
**Predict**: Each matchup 

In [1]:
# NumPy for numerical computing
import numpy as np

In [ ]:
#read in the team data
df_teams = pd.read_csv('../../data/raw/MDataFiles_Stage2/MTeams.csv')

The MTeams.csv data has the team name, the ID number which is a 4 digit ID, the first season they were in D1 and the last season they were in D1. If the last season date is 2021, they are currently in D1. The dataset goes back to 1985 so that's the earliest year available.

In [ ]:
df_teams.head()

In [ ]:
#read in the seasons data
df_seasons = pd.read_csv('../../data/raw/MDataFiles_Stage2/MSeasons.csv')

The MSeasons.csv data has the season (year) that the tournament was played, DayZero is the 0th day of the season so that the men's championship game is on day 154, whichever region comes first alphabetically is RegionW, RegionX is the region that plays RegionW in the semifinals, in the leftover 2 regions the first alphabetically is RegionY, RegionZ plays RegionY in the semifinals.

In [ ]:
df_seasons.head()

In [ ]:
#read in the tournament seed data data
df_seeds = pd.read_csv('../../data/raw/MDataFiles_Stage2/MNCAATourneySeeds.csv')

The MNCAATourneySeeds.csv data has the season (year) the tournament was played, the team ID from MTeams.csv (df_teams), and the seed. The seed is given with the first letter (W,X,Y,Z) to specify the region and a number 01 - 16 for the seed. Play-in teams will also have an 'a' or a 'b' to distinguish which team was ranked lower ('b').

In [ ]:
df_seeds.head()

In [ ]:
#read in the regular season results data
df_regseason = pd.read_csv('../../data/raw/MDataFiles_Stage2/MRegularSeasonCompactResults.csv')

The MRegularSeasonCompactResults.csv data has the season (year) of the game, DayNum is which day of the season that the game occured (this can be combined with DayZero of the MSeasons.csv (df_seasons) data to get an exact date), WTeamID is the team ID (MTeams.csv / df_teams) of the winning team, WScore is the score of the winning team, LTeamID is the team ID of the losing team, LScore is the score of the losing Team, NumOT is the number of overtimes played in that game, and WLoc is the location of the winning team (H stands for home, A stands for away, and N is a neutral site).

In [ ]:
df_regseason.head()

In [ ]:
#read in the tournament results data
df_tourneyresults = pd.read_csv('../../data/raw/MDataFiles_Stage2/MNCAATourneyCompactResults.csv')

The MNCAATourneyCompactResults.csv data is formatted exactly the same way as the regular season results data (MRegularSeasonCompactResults.csv / df_regseason). The site is always neutral and therefore N. DayNum 134/135 are the play-in games, DayNum 136/137 are Round 1, DayNum 138/139 are Round 2, DayNum 143/144 are Round 3 (sweet sixteen), DayNum 145/146 are Round 4 (elite eight), DayNum 152 is final four, and DayNum 154 is the championship game. 

In [ ]:
df_tourneyresults.head()

In [ ]:
#read in the regular season detailed results data
df_regseasonDets = pd.read_csv('../../data/raw/MDataFiles_Stage2/MRegularSeasonDetailedResults.csv')

The MRegularSeasonDetailedResults.csv data has the game by game details of regular season games since the 2002-2003 season. The first 8 columns are the same as in the compact results file (MRegularSeasonCompactResults.csv / df_regseason). WFGM is the # of field goals made by the winning team, WFGA is the # of field goals attempted by the winning team. WFGM3 is the 3 pointers made by the winning team, WFGA3 is the # of 3 pointers attempted. WFTM is the # of free throws made by the winning team, WFTA is the # of free throws attempted by the winning team. WOR is the # of offensive rebounds pulled by the winning team, WDR is the # of defensive rebounds pulled by the winning team. WAst is the # of assists by the winning team. WTO is the # of turnovers committed by the winning team. WStl is the # of steals by the winning team, WBlk is the # of blocks by the winning team. WPF is the # of personal fouls committed by the winning team. Then there are the same stats but with L instead of W for the losing team.

In [ ]:
df_regseasonDets.head()

In [ ]:
#read in the tournament detailed results data
df_tourneyDets = pd.read_csv('../../data/raw/MDataFiles_Stage2/MNCAATourneyDetailedResults.csv')

MNCAATourneyDetailedResults.csv has the same information as the detailed regular seadon data but for the tournament.

In [ ]:
df_tourneyDets.head()

In [ ]:
#read in the cities data
df_cities = pd.read_csv('../../data/raw/MDataFiles_Stage2/cities.csv')

cities.csv has the data for the cities where the games were held. CityID is a unique 4 digit number for the city, City is the text name of the city and State is the state the citie is in.

In [ ]:
df_cities.head()

In [ ]:
#read in the games city data
df_gamecities = pd.read_csv('../../data/raw/MDataFiles_Stage2/MGameCities.csv')

MGameCities.csv has all the game cities data since 2010. Season, DayNum, WTeamID, LTeamID, and CityID have been discussed before. CRType is either Regular for a regular season game, NCAA is a tournament game, and Secondary is a game from the secondary data.

In [ ]:
df_gamecities.head()

In [ ]:
#read in the rankings data
df_rankings = pd.read_csv('../../data/raw/MDataFiles_Stage2/MMasseyOrdinals.csv')

The MMasseyOrdinals.csv has ranking information from as far back as 2002. Season and TeamID have already been discussed. RankingDayNum has the day number that it is appropriate to use the ranking information for predicting games. For example, you shouldn't use a ranking on day 110 to predict a game on day 100. SystemName is a 3 letter abbreviation for the ranking system. OrdinalRank is the ranking of the teams on that day in that system. By convention the final ranking before the tournament is RankingDayNum = 133.

In [ ]:
df_rankings.head()

In [ ]:
#read in the tournament slot data
df_slots = pd.read_csv('../../data/raw/MDataFiles_Stage2/MNCAATourneySlots.csv')

MNCAATourneySlots.csv data can be used to identify what round a historical tournament game occured in and what seed/slot the teams that played that game were. This could be useful for simulating the progression of a bracket. Slot uniquely identifies one of the tournament games. For play-in games, it is a three-character string identifying the seed fulfilled by the winning team, such as W16 or Z13. For regular tournament games, it is a four-character string, where the first two characters tell you which round the game is (R1, R2, R3, R4, R5, or R6) and the second two characters tell you the expected seed of the favored team. For example, the R2W1 slot indicates the Round 2 game that would have the 1 seed from the W bracket, assuming that all favored teams have won up to that point. Even if that R2W1 slot were actually a game between the W09 and W16 teams, it is still considered to be the R2W1 slot. The slot names are different for the final two rounds, where R5WX identifies the national semifinal game between the winners of regions W and X, and R5YZ identifies the national semifinal game between the winners of regions Y and Z, and R6CH identifies the championship game.

StrongSeed indicates the expected stronger-seeded team that plays in this game. For Round 1 games, a team seed is identified in this column (as listed in the "Seed" column in the MNCAATourneySeeds.csv file), whereas for subsequent games, a slot is identified in this column. In the first record of this file (slot R1W1), we see that seed W01 is the "StrongSeed", which during the 1985 tournament would have been Georgetown. Whereas for games from Round 2 or later, rather than a team seed, we will see a "slot" referenced in this column. So in the 33rd record of this file (slot R2W1), it tells us that the winners of slots R1W1 and R1W8 will face each other in Round 2. Of course, in the last few games of the tournament - the national semifinals and finals - it's not really meaningful to talk about a "strong seed" or "weak seed", since you would have #1 seeds favored to face each other, but those games are nevertheless represented in the same format for the sake of consistency. WeakSeed indicates the expected weaker-seeded team that plays in this game, assuming all favored teams have won so far. For Round 1 games, a team seed is identified in this column (as listed in the "Seed" column in the MNCAATourneySeeds.csv file), whereas for subsequent games, a slot is identified in this column.

In [ ]:
df_slots.head()

In [ ]:
#read in the tournament slot data
df_seed = pd.read_csv('../../data/raw/MDataFiles_Stage2/MNCAATourneySeedRoundSlots.csv')

MNCAATourneySeedRoundSlots.csv represents the bracket structure. Seed is the tournament seed of the team. GameRound is the round during the tournament that the game would occur in, where Round 0 (zero) is for the play-in games, Rounds 1/2 are for the first weekend, Rounds 3/4 are for the second weekend, and Rounds 5/6 are the national semifinals and finals. GameSlot is the game slot that the team would be playing in, during the given GameRound. The naming convention for slots is the same as in df_slots. EarlyDayNum and LateDayNum describe the earliest possible, and latest possible, DayNums that the game might be played on.

In [ ]:
df_seed.head()

There is a lot of data that I don't plan to use here. I want to take the regular season data of wins and losses, as well as the detailed regular season information, and the tournament seeds to predict the tournament results. So now I need to make that dataframe with just that information.

In [ ]:
df = pd.DataFrame()

The first thing I want to do is a little feature engineering. The detailed regular season data has stats for each game which is great. I'm going to want to turn these per game stats into averages per season. So I'll take the TeamID and compute the averages for that team. 

In [ ]:
df_regseasonDets.head()

Let's start by turning all the attempts and made shots into a percentage.

In [ ]:
df_regseasonDets['WFGP'] = df_regseasonDets.WFGM / df_regseasonDets.WFGA
df_regseasonDets['WFGP3'] = df_regseasonDets.WFGM3 / df_regseasonDets.WFGA3
df_regseasonDets['WFTP'] = df_regseasonDets.WFTM / df_regseasonDets.WFTA
df_regseasonDets['LFGP'] = df_regseasonDets.LFGM / df_regseasonDets.LFGA
df_regseasonDets['LFGP3'] = df_regseasonDets.LFGM3 / df_regseasonDets.LFGA3
df_regseasonDets['LFTP'] = df_regseasonDets.LFTM / df_regseasonDets.LFTA

In [ ]:
df_regseasonDets.head()

Let's create a dataframe for that has the average information of the games grouped by season and TeamID. I'm going to do this by first making two different dataframes, one for the winning team stats, and one for the losing. Then I'll concatenate them together and then take the averages. 

In [ ]:
df_WregseasonDets = pd.DataFrame() 
df_WregseasonDets['TeamID'] = df_regseasonDets.WTeamID
df_WregseasonDets['Season'] = df_regseasonDets.Season
df_WregseasonDets['Score'] = df_regseasonDets.WScore
df_WregseasonDets['FGP'] = df_regseasonDets.WFGP
df_WregseasonDets['FGP3'] = df_regseasonDets.WFGP3
df_WregseasonDets['FTP'] = df_regseasonDets.WFTP
df_WregseasonDets['OR'] = df_regseasonDets.WOR
df_WregseasonDets['DR'] = df_regseasonDets.WDR
df_WregseasonDets['Ast'] = df_regseasonDets.WAst
df_WregseasonDets['TO'] = df_regseasonDets.WTO
df_WregseasonDets['Blk'] = df_regseasonDets.WBlk
df_WregseasonDets['Stl'] = df_regseasonDets.WStl
df_WregseasonDets['PF'] = df_regseasonDets.WPF

In [ ]:
df_WregseasonDets.head()

In [ ]:
df_WregseasonDets.shape

In [ ]:
df_LregseasonDets = pd.DataFrame() 
df_LregseasonDets['TeamID'] = df_regseasonDets.LTeamID
df_LregseasonDets['Season'] = df_regseasonDets.Season
df_LregseasonDets['Score'] = df_regseasonDets.LScore
df_LregseasonDets['FGP'] = df_regseasonDets.LFGP
df_LregseasonDets['FGP3'] = df_regseasonDets.LFGP3
df_LregseasonDets['FTP'] = df_regseasonDets.LFTP
df_LregseasonDets['OR'] = df_regseasonDets.LOR
df_LregseasonDets['DR'] = df_regseasonDets.LDR
df_LregseasonDets['Ast'] = df_regseasonDets.LAst
df_LregseasonDets['TO'] = df_regseasonDets.LTO
df_LregseasonDets['Blk'] = df_regseasonDets.LBlk
df_LregseasonDets['Stl'] = df_regseasonDets.LStl
df_LregseasonDets['PF'] = df_regseasonDets.LPF

In [ ]:
df_LregseasonDets.head()

In [ ]:
df_LregseasonDets.shape

Concatenate df_WregseasonDets and df_LregseasonDets together.

In [ ]:
df_regseasonDets_teams = pd.DataFrame()
df_regseasonDets_teams = pd.concat([df_WregseasonDets, df_LregseasonDets])

In [ ]:
df_regseasonDets_teams.shape

In [ ]:
df_regseasonDets_teams.head()

Take the averages by team and season.

In [ ]:
df = df_regseasonDets_teams.groupby(['Season','TeamID']).agg({'Score':'mean','FGP':'mean',
                                                       'FGP3':'mean','FTP':'mean',
                                                       'OR':'mean','DR':'mean','Ast':'mean',
                                                      'TO':'mean','Stl':'mean','Blk':'mean',
                                                      'PF':'mean'})
df.head()

In [ ]:
df.shape

I would also like to add the rankings before the tournament. This is day 133.

In [ ]:
df_rankings.head()

In [ ]:
last_day = df_rankings['RankingDayNum']==133

In [ ]:
df_rankings_last = df_rankings[last_day]

In [ ]:
df_rankings_last.head()

In [ ]:
df_rankings_last.SystemName.unique()

Ok there are a lot of ranking systems here and I definitely do not want all of them. So for each team and year let's take the mean. This might help to give us an idea of how each team is ranked. I might end up just taking the seed and not caring about the rank if I don't think this is helpful information. 

In [ ]:
df_rankings_last_Ave  = df_rankings_last.groupby(['Season','TeamID']).agg({'OrdinalRank':['mean']})#,'RankMin':'min','RankMax':'max'})
df_rankings_last_Ave.head()

In [ ]:
df_rankings_last_Ave.shape

In [ ]:
frames = [df,df_rankings_last_Ave]
df = pd.concat(frames, axis=1)

In [ ]:
df.head()

I don't love this ranking column name so I'll just rename it rank. Remember this is the mean rank.

In [ ]:
list(df.columns.values)

In [ ]:
df.rename(columns = {('OrdinalRank', 'mean'):'Rank'}, inplace = True) 

In [ ]:
df.head()

Right now Season and TeamID are indexes and it's kicking my ass so I'm going to make them columns. 

In [ ]:
df.reset_index(inplace=True)

In [ ]:
df.head()

Now let's add the seed to our data. Since we only have regular season data from season 2002, we're going to only use seed data from then as well.

In [ ]:
df_seeds.head()

In [ ]:
seeds_mask = df_seeds.Season>2002

In [ ]:
df_seeds2002 = df_seeds[seeds_mask]

In [ ]:
df = df.merge(df_seeds2002, how = 'inner', on = ['Season', 'TeamID'])

In [ ]:
df.head()

Now that the seed is there, let's turn it into an integer by removing the region. I don't think we'll need the region data since we have the team IDs but if we do, I can always add it back in.

In [ ]:
df['Seed'] = df['Seed'].str[1:]

In [ ]:
df.head()

Since we just stripped a letter from our Seed, the seed is probably still a string. So let's verify that and change it to a float. We have all of those pesky play in games as well. So for our purpose I'm changing 11a to 11.1 and 11b to 11.2 so they'll be numerical. I'm not meaning to imply that one team is ranked higher than another. If we need to change this in the future we can.

In [ ]:
df.Seed.replace(['11a','11b','12a','12b','13a','13b','14a','14b','16a','16b'],
['11','11','12','12','13','13','14','14','16','16'],inplace=True)

In [ ]:
df['Seed'] = df['Seed'].astype(float)

In [ ]:
df.dtypes

Perfect. I'm happy with this and I'll save the data as a csv file in the processed data folder.

In [ ]:
df.to_csv('../../data/processed/data_averages_2021.csv',index=False)